# Baixando arquivos do github e e extraindo em um arquivo csv

In [ ]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    # Cria a pasta datasets/housing
    os.makedirs(housing_path, exist_ok=True)

    # Cria um caminho absoluto para o arquivo .tgz
    tgz_path = os.path.join(housing_path, "housing.tgz")

    # Baixa o arquivo .tgz e joga no caminho criado
    urllib.request.urlretrieve(housing_url, tgz_path)

    # Abre o arquivo, extrai para datasets/housing e depois fecha ele
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path, filter="data")
    housing_tgz.close()

fetch_housing_data()

# Carregando dados em um dataframe

In [ ]:
# Carregando dados
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

# Obtendo detalhes do dataframe

In [ ]:
""" # Obtendo uma rápida descricao dos dados
housing.info()

# Descobrindo a contagem de cada tipo de dado na coluna ocean_proximity
housing["ocean_proximity"]

# Dá um resumo dos atributos numéricos do dataframe
housing.describe() """

# Plotando os dados usando gráficos

In [ ]:
""" %matplotlib inline
import matplotlib.pyplot as plt 
housing.hist(bins=50, figsize=(20,15)) """

# Criando um conjunto de testes (método 1)
Esse método é menos complexo, porém, toda vez que executarmos o programa, ele selecionará elementos diferentes (random).\
Em pouco tempo, todos os elementos de teste, passarao pelo conjunto de treinamento. E nao queremos isso!

In [ ]:
""" import numpy as np

def split_train_test(data, test_ratio):
    # Retorna um array de inteiros de 0 até len(data)
    shuffled_indices = np.random.permutation(len(data))

    # Calcula a quantidade de elementos que farao parte do conjunto de teste
    test_set_size = int(len(data) * test_ratio)

    # Pega as posicoes dos elementos de teste como sendo do array embaralhado do inicio até test_set_size
    test_indices = shuffled_indices[:test_set_size]

    # Pega as posicoes dos elementos de treinamento como sendo de test_set_size até o fim do array embaralhado
    train_indices = shuffled_indices[test_set_size:]

    # dataframe.iloc voce pode passar tambem um array de indices e ele retornara um dataframe mostrando aqueles elementos
    return data.iloc[train_indices], data.iloc[test_indices]

# Geralmente para treinamento se utiliza 20% do conjunto total
train_set, test_set = split_train_test(housing, 0.2)

print(len(train_set))

print(len(test_set))
 """

# Criando um conjunto de testes (método 2)
Esse método consiste em calcular o hash do identificador de cada instancia e verificar se é menor que 20% do valor máximo dele.\
Assim, o conjunto de testes consistente em todas as execucoes do programa, mesmo que sejam adicionadas instancias novas.

In [ ]:
""" import numpy as np
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * (2 ** 32)

def split_train_set_by_id(data, test_ratio, id_column):
    # Pega uma series contendo os ids de cada entrada 
    ids = data[id_column]

    # apply() recebe uma funcao para aplicar a cada elemento do conjunto dem ids
    is_in_test_set = ids.apply(lambda id: test_set_check(id, test_ratio))

    # dataframe.loc[] serve para retornar as colunas como series. Se usar .loc[['col1', 'col2']] retorna um dataframe
    # se passar um array de boolean (array.length == len(dataframe)) para o .loc[], ele retorna o elemento i caso array[i] == True
    # o til (~) antes do array is_in_test_set serve para negar elemento por elemento, ou seja, se array = [True, True, False], logo ~array[False, False, True]
    # testar com iloc depois
    return data.loc[~is_in_test_set], data.loc[is_in_test_set]

# Esse conjunto de dados nao tem uma coluna de ID, logo vamos improvisar uma:
housing_with_id = housing.reset_index() # adiciona um index a coluna
train_set, test_set = split_train_set_by_id(housing, 0.2, "index")
 """

# Criando um conjunto de testes (método 3)
Esse método utiliza o método train_test_split do sklearn

In [ ]:
""" # Basicamente esse método faz o que implmentamos manualmente no método 2
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42) # 42 é o numero da sorte :)
 """

# Adicionando uma categoria com estratos
Vamos adicionar uma nova categoria ao dataframe a partir da estratificacao da renda media.\
Se voce quiser fazer uma pesquisa representando a populacao dos EUA (51,3% M, 48,7% H) voce deve seguir a mesma proporcao ao tomar um conjunto de pesquisa.\
Da mesma forma fazemos ao tomar um conjunto de testes e treinamento. Assim evitamos o *viés de amostagem*.\
Nesse caso, a categoria Homem é um estrato, e Mulher outro.\
Da mesma maneira, vamos criar estratos baseados na renda média.

In [ ]:
import pandas as pd
import numpy as np

# O método cut subdivide uma coluna em grupos menores de acordo com a configuracao
housing["income_cat"] = pd.cut(housing["median_income"], # a partir de qual coluna
                               bins=[0., 1.5, 3.0, 4.5, 6, np.inf], # subdivisao que vemos quando plotamos a coluna
                               labels=[1, 2, 3, 4, 5]) # qual a categoria

#housing["income_cat"].hist()

from sklearn.model_selection import StratifiedShuffleSplit

def stratified_split():
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) # n_splits = numero de divisoes

    # como nesse caso n_splits=1 na primeira iteracao ja podemos retornar os conjuntos na primeira iteracao
    for train_indices, test_indices in split.split(housing, housing["income_cat"]):
        strat_train_set = housing.loc[train_indices]
        strat_test_set = housing.loc[test_indices]

        print(strat_train_set.info())

        # Agora que já separamos os dados em categorias, podemos remover a coluna income_cat
        strat_train_set.drop("income_cat", axis=1, inplace=True)
        strat_test_set.drop("income_cat", axis=1, inplace=True)

        #print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))
        return strat_train_set, strat_test_set


# Redefinindo o conjunto de estudos
A partir de agora vamos assumir que housing é apenas o conjunto de treinamento.\
Já que ele está corretamente estratificado, a amostragem será identica ao de todo o conjunto.\
Fazemos isso para evitar de trabalhar com o conjunto de testes.\

In [ ]:
strat_train_set, strat_test_set = stratified_split()
housing = strat_train_set.copy()

# Visualizando os dados geográficos

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)

# Buscando correlacoes
